## Introduction

This notebook is dedicated to developing a robust machine learning model capable of accurately predicting Formula 1 race ranks, providing valuable insights for F1 enthusiasts and strategic analysis.

Building upon the comprehensive data exploration and feature engineering from `f1_data_exploration.ipynb`, we consolidate relevant race and qualifying data into a unified pandas DataFrame. This dataset is then meticulously cleaned and prepared, leveraging a diverse set of features encompassing driver and constructor historical performance, qualifying results, circuit characteristics, and more, to capture the complex dynamics of F1 racing.

The core of this notebook focuses on identifying the most suitable machine learning model and its respective hyperparameters. We will systematically explore `RandomForestRegressor` and `GradientBoostingRegressor` using advanced cross-validation and hyperparameter tuning techniques from `sklearn.model_selection`. The optimization process will prioritize not just the accuracy of predicted F1 race results, but also the efficiency and generalization of the training procedure. Model performance will be rigorously evaluated using metrics such as Mean Absolute Error (MAE) and R-squared, with a particular focus on how well the model predicts the actual finishing order.

Finally, the best-performing model, along with its optimized parameters, will be serialized and saved as `f1_rank_predictor.pkl` in the `models/` directory, ready for seamless integration into the backend prediction service.

## Imports and setup

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib
import fastf1
import os

## Loading Race Data

Race data for years 2015 to 2025 are loaded into `notebooks\data_cache`.

In [21]:
import fastf1
import pandas as pd
import os

# --- 1. Configure FastF1 Caching ---
# Ensure this directory exists and is accessible
cache_dir = './data_cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    print(f"Created cache directory: {cache_dir}")
else:
    print(f"Cache directory already exists: {cache_dir}")

fastf1.Cache.enable_cache(cache_dir)
print(f"FastF1 caching enabled to '{cache_dir}'")

# --- IMPORTANT: REMOVE OR COMMENT OUT THIS LINE TO SEE ALL FASTF1 MESSAGES ---
# fastf1.set_log_level('ERROR') # <--- Make sure this line is commented out or deleted!
print("\nFastF1 log level set to default (INFO) to show all messages.")


# --- 2. Define the Years You Want to Read ---
start_year = 2015
end_year = 2025 # Includes this year

# Lists to hold DataFrames from each race
all_race_results = []
all_qualifying_results = []

print(f"\n--- Starting data collection for seasons {start_year} to {end_year} ---")

for year in range(start_year, end_year + 1):
    print(f"\nProcessing season: {year}")
    try:
        # Get the event schedule for the current year
        print(f"  Attempting to get event schedule for {year}...")
        schedule = fastf1.get_event_schedule(year)
        print(f"  Schedule loaded for {year}. Found {len(schedule)} events.")

        # Filter for actual race events (excluding test sessions, etc.)
        race_events = schedule.loc[schedule['EventFormat'].isin(['conventional', 'sprint'])]
        print(f"  Filtered for 'conventional'/'sprint' race events. Found {len(race_events)} race weekends.")

        if race_events.empty:
            print(f"  No 'conventional' or 'sprint' race events found for {year}. Skipping season.")
            continue

        for round_num in race_events['RoundNumber']:
            print(f"  -> Attempting to load data for {year} Round {round_num}...")
            try:
                # --- Load Race Session Results ---
                race_session_name = f"{year} Round {round_num} (Race)"
                print(f"    Loading {race_session_name}...")
                race_session = fastf1.get_session(year, round_num, 'R')
                race_session.load(telemetry=False, laps=False, weather=False)

                if race_session.results.empty:
                    print(f"    WARNING: No results found for {race_session_name}. It might not have happened yet or data is unavailable.")
                else:
                    results_df = race_session.results.copy()
                    results_df['Season'] = year
                    results_df['Round'] = round_num
                    results_df['EventName'] = race_session.event['EventName']
                    results_df['SessionType'] = 'Race'
                    all_race_results.append(results_df)
                    print(f"    Loaded {len(results_df)} race results entries for {race_session_name}.")

                # --- Optionally, load Qualifying Session Results ---
                try:
                    quali_session_name = f"{year} Round {round_num} (Qualifying)"
                    print(f"    Loading {quali_session_name}...")
                    quali_session = fastf1.get_session(year, round_num, 'Q')
                    quali_session.load(telemetry=False, laps=False, weather=False)

                    if quali_session.results.empty:
                        print(f"    WARNING: No results found for {quali_session_name}. It might not have happened yet or data is unavailable.")
                    else:
                        quali_results_df = quali_session.results.copy()
                        quali_results_df['Season'] = year
                        quali_results_df['Round'] = round_num
                        quali_results_df['EventName'] = quali_session.event['EventName']
                        quali_results_df['SessionType'] = 'Qualifying'
                        all_qualifying_results.append(quali_results_df)
                        print(f"    Loaded {len(quali_results_df)} qualifying results entries for {quali_session_name}.")

                except Exception as e:
                    print(f"    Error loading Qualifying for {year} R{round_num}: {e}. Skipping qualifying data for this race.")

            except Exception as e:
                print(f"  CRITICAL ERROR loading session {year} R{round_num}: {e}. Skipping this race and round.")

    except Exception as e:
        print(f"Error retrieving schedule for {year}: {e}. This season might not exist or there's a connectivity issue. Skipping season.")

# --- 3. Concatenate All DataFrames ---
print("\n--- Concatenating collected data ---")
if all_race_results:
    full_race_results_df = pd.concat(all_race_results, ignore_index=True)
    print("\n--- Full Race Results DataFrame ---")
    print(f"Total entries in Race Results: {len(full_race_results_df)}")
    print(full_race_results_df.head())
    print(full_race_results_df.info())
else:
    print("\nNo race results data collected into a DataFrame.")

if all_qualifying_results:
    full_quali_results_df = pd.concat(all_qualifying_results, ignore_index=True)
    print("\n--- Full Qualifying Results DataFrame ---")
    print(f"Total entries in Qualifying Results: {len(full_quali_results_df)}")
    print(full_quali_results_df.head())
    print(full_quali_results_df.info())
else:
    print("\nNo qualifying results data collected into a DataFrame.")

print("\n--- Data collection process finished ---")

Cache directory already exists: ./data_cache
FastF1 caching enabled to './data_cache'

FastF1 log level set to default (INFO) to show all messages.

--- Starting data collection for seasons 2015 to 2025 ---

Processing season: 2015
  Attempting to get event schedule for 2015...


core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


  Schedule loaded for 2015. Found 19 events.
  Filtered for 'conventional'/'sprint' race events. Found 19 race weekends.
  -> Attempting to load data for 2015 Round 1...
    Loading 2015 Round 1 (Race)...


logger      WARNING 	Failed to load session info data!
core        WARNING 	Cannot load laps, telemetry, weather, and message data because the relevant API is not supported for this session.
core           INFO 	Finished loading data for 18 drivers: ['44', '6', '5', '19', '12', '3', '27', '9', '55', '11', '22', '7', '33', '8', '13', '26', '20', '77']
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


    Loaded 18 race results entries for 2015 Round 1 (Race).
    Loading 2015 Round 1 (Qualifying)...


...

Error retrieving schedule for 2023: Failed to load any schedule data.. This season might not exist or there's a connectivity issue. Skipping season.

Processing season: 2024
  Attempting to get event schedule for 2024...


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


Error retrieving schedule for 2024: Failed to load any schedule data.. This season might not exist or there's a connectivity issue. Skipping season.

Processing season: 2025
  Attempting to get event schedule for 2025...


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


Error retrieving schedule for 2025: Failed to load any schedule data.. This season might not exist or there's a connectivity issue. Skipping season.

--- Concatenating collected data ---

--- Full Race Results DataFrame ---
Total entries in Race Results: 1000
  DriverNumber BroadcastName Abbreviation  DriverId  TeamName TeamColor  \
0           44                        HAM  hamilton  Mercedes             
1            6                        ROS   rosberg  Mercedes             
2            5                        VET    vettel   Ferrari             
3           19                        MAS     massa  Williams             
4           12                        NAS      nasr    Sauber             

     TeamId  FirstName  LastName          FullName  ...  Q2  Q3  \
0  mercedes      Lewis  Hamilton    Lewis Hamilton  ... NaT NaT   
1  mercedes       Nico   Rosberg      Nico Rosberg  ... NaT NaT   
2   ferrari  Sebastian    Vettel  Sebastian Vettel  ... NaT NaT   
3  williams     Felip

## Data Preprocessing
We now handle any missing values in the dataframe, ensuring appropriate format for training.
The code cell below reveals the columns in `full_race_results_df`, from which we will remove the ones unnecessary in the training of the model.

In [22]:
full_race_results_df.columns

Index(['DriverNumber', 'BroadcastName', 'Abbreviation', 'DriverId', 'TeamName',
       'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName',
       'HeadshotUrl', 'CountryCode', 'Position', 'ClassifiedPosition',
       'GridPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps',
       'Season', 'Round', 'EventName', 'SessionType'],
      dtype='object')

The columns `'DriverNumber', 'BroadcastName', 'DriverId', 'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName', 'HeadshotUrl', 'CountryCode', 'ClassifiedPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps', 'SessionType'` are dropped from the original DataFrame, and the resulting DataFrame is stored in `new_df`.

As the model will be made to predict the raw `Position` of each driver, `ClassifiedPosition` is removed since race retirement is extremely situational and `Position` will instead be used as the target variable.

In [23]:
race_data_for_merge = full_race_results_df.drop(['DriverNumber', 'BroadcastName', 'DriverId', 'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName', 'HeadshotUrl', 'CountryCode', 'ClassifiedPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps', 'SessionType'], axis=1)

`'Q1', 'Q2', and 'Q3'` are `timedelta` objects and are converted into seconds and stored respectively in the new columns `'Q1_s', 'Q2_s', and 'Q3_s'`. This will allow for its meaningful use when training the model.

In [24]:
# Creating new columns to hold values of 'Q1', 'Q2, and 'Q3' in seconds.
full_quali_results_df['Q1_s'] = full_quali_results_df['Q1'].dt.total_seconds().fillna(9999.0)
full_quali_results_df['Q2_s'] = full_quali_results_df['Q2'].dt.total_seconds().fillna(9999.0)
full_quali_results_df['Q3_s'] = full_quali_results_df['Q3'].dt.total_seconds().fillna(9999.0)

Since certain qualifying data are useful for training the model, we merge a subset of the 2 dataframes `full_race_results_df` and `full_quali_results_df`, keeping only the relevant information.

In [25]:
quali_data_for_merge = full_quali_results_df[['Abbreviation', 'Season', 'Round', 'EventName', 'Q1_s', 'Q2_s', 'Q3_s']].copy() 

merged_df = pd.merge(
    race_data_for_merge,
    quali_data_for_merge,
    on=['Abbreviation', 'Season', 'Round', 'EventName'],
    how='left'
)

# Organising columns for readability and easier management
merged_df = merged_df[[    
    'Season',         
    'Round',
    'EventName',
    'Abbreviation',   # Driver identifier eg. 'VER', 'PER', 'ALO'
    'TeamName',      
    'GridPosition',   # Qualifying positions, also positions in which they start on during race
    'Q1_s',           # Q1 time in seconds
    'Q2_s',
    'Q3_s',
    'Position'        # The target variable for race prediction
]]

In [26]:
quali_data_for_merge = full_quali_results_df[['Abbreviation', 'Season', 'Round', 'EventName', 'Q1_s', 'Q2_s', 'Q3_s']].copy()

merged_df = pd.merge(
    race_data_for_merge,
    quali_data_for_merge,
    on=['Abbreviation', 'Season', 'Round', 'EventName'],
    how='left'
)

# Organising columns for readability and easier management
merged_df = merged_df[[
    'Season', 
    'Round',
    'EventName',
    'Abbreviation', # Driver identifier eg. 'VER', 'PER', 'ALO'
    'TeamName',
    'GridPosition', # Qualifying positions, also positions in which they start on during race
    'Q1_s', # Q1 time in seconds
    'Q2_s',
    'Q3_s',
    'Position' # The target variable for race prediction
]]

merged_df.head(20) # DataFrame snippet of race 1

,Season,Round,EventName,Abbreviation,TeamName,GridPosition,Q1_s,Q2_s,Q3_s,Position
0,2015,1,Australian Grand Prix,HAM,Mercedes,1.0,88.586,86.894,86.327,1.0
1,2015,1,Australian Grand Prix,ROS,Mercedes,2.0,88.906,87.097,86.921,2.0
2,2015,1,Australian Grand Prix,VET,Ferrari,4.0,89.307,87.742,87.757,3.0
3,2015,1,Australian Grand Prix,MAS,Williams,3.0,89.246,87.895,87.718,4.0
4,2015,1,Australian Grand Prix,NAS,Sauber,10.0,90.430,88.800,9999.000,5.0
5,2015,1,Australian Grand Prix,RIC,Red Bull,6.0,89.788,88.679,88.329,6.0
6,2015,1,Australian Grand Prix,HUL,Force India,13.0,89.651,89.208,9999.000,7.0
7,2015,1,Australian Grand Prix,ERI,Sauber,15.0,91.376,9999.000,9999.000,8.0
8,2015,1,Australian Grand Prix,SAI,Toro Rosso,7.0,89.597,88.601,88.510,9.0
9,2015,1,Australian Grand Prix,PER,Force India,14.0,89.990,89.209,9999.000,10.0


During the 2023 Singapore Grand Prix, driver Lance Stroll was withdrawn from the Sunday race following a heavy crash during Saturday's qualifying session. 

As a result, we see an empty value for Stroll's `'GridPosition'` and `'Position'`for the event.

As there are instances (rows 197, 241, 278, 633) where drivers starting from the pitlanes has resulted in `'GridPosition'` and `'Position'` holding the value of 0.0, we deduce that this is an anomaly on the side of the API.

In [ ]:
# Result should be 1, referring to this singular anomaly
print("Number of rows with missing values for 'GridPosition' before cleaning:", merged_df['GridPosition'].isna().sum())

# A simple fix would be to replace all missing values with 0.0
merged_df.loc[merged_df['GridPosition'].isna(), 'GridPosition'] = 0.0
merged_df.loc[merged_df['Position'].isna(), 'Position'] = 0.0
merged_df.loc[merged_df['Q1_s'].isna(), 'Q1_s'] = 0.0
merged_df.loc[merged_df['Q2_s'].isna(), 'Q2_s'] = 0.0
merged_df.loc[merged_df['Q3_s'].isna(), 'Q3_s'] = 0.0

# Result after cleaning
print("Number of rows with missing values for 'GridPosition' after cleaning:", merged_df['GridPosition'].isna().sum())

#merged_df.to_csv(os.path.join(os.getcwd(), 'merged_df.csv'), index=False) # This optional line creates a .csv file for better visualisation

Number of rows with missing values for 'GridPosition' before cleaning: 0
Number of rows with missing values for 'GridPosition' after cleaning: 0


#### One-hot Encoding

Now that we have the prepared `merged_df`, we have to convert categorical (non-numerical) data into a numerical format that machine learning algorithms can understand and process effectively. 

Let us look at the `'TeamName'` column. Without one-hot encoding, if we were to simply assign each team values eg. Red Bull = 1, Ferrari = 2, Mercedes = 3, a machine learning model would interpret these numbers as having an inherent order or magnitude. It might think that Mercedes (3) is "better" or "more important" or "further away" from Red Bull (1) than Ferrari (2) is. This is clearly false for nominal categories like team names, where there's no inherent numerical relationship. This false ordinality can mislead the model and lead to incorrect predictions or reduced performance.

With one-hot encoding, we create new binary (0 or 1) columns for every team that exists. If in row 1 was Max Verstappen from team Red bull Racing, we would now have a value of 1 in Team_RedBull and a 0 in the columns of every other team.

In [ ]:
# Define categorical columns for one-hot encoding
categorical_cols_to_encode = ['Abbreviation', 'TeamName', 'EventName']

# Perform One-Hot Encoding
# drop_first=True prevents multicollinearity; dtype=int ensures binary integers
merged_df_encoded = pd.get_dummies(
    merged_df,
    columns=categorical_cols_to_encode,
    drop_first=True,
    dtype=int
)

# Note the increase in number of columns after encoding
print(f"\nOriginal DataFrame shape: {merged_df.shape}")
print(f"Encoded DataFrame shape: {merged_df_encoded.shape}")

#merged_df_encoded.to_csv(os.path.join(os.getcwd(), 'merged_df_encoded.csv'), index=False) # This optional line creates a .csv file for better visualisation


Original DataFrame shape: (1000, 10)
Encoded DataFrame shape: (1000, 68)


## Model Selection

Splitting data into features (X) and the target variable (y).

In [40]:
# Define the target variable
y = merged_df_encoded['Position']

# Define the features (X) by dropping the target and any other non-feature columns
columns_to_exclude_from_X = ['Position'] # Only exclude the target variable

X = merged_df_encoded.drop(columns=columns_to_exclude_from_X, axis=1)

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")


X shape: (1000, 67)
y shape: (1000,)


Dividing the data into training and testing sets.

In [41]:
from sklearn.model_selection import train_test_split

# Split the data with 80% for training and 20% for testing
# random_state ensures reproducibility of your split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTrain set shape (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Test set shape (X_test, y_test): {X_test.shape}, {y_test.shape}")


Train set shape (X_train, y_train): (800, 67), (800,)
Test set shape (X_test, y_test): (200, 67), (200,)


Choose __Candidate Models__ and define __Hyperparameter Grids__

In [42]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
import joblib # To save the model

# Define the scoring metric for optimization
# For MAE, lower is better, so we use 'neg_mean_absolute_error' for GridSearchCV
# which maximizes the score.
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# --- Model 1: RandomForestRegressor ---
rf_model = RandomForestRegressor(random_state=42)
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': [0.6, 0.8, 1.0],
    'min_samples_leaf': [1, 2, 4],
}

# --- Model 2: GradientBoostingRegressor ---
gb_model = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200, 300, 700],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
}

# You could add more models here, e.g.,
# from xgboost import XGBRegressor
# xgb_model = XGBRegressor(random_state=42)
# xgb_param_grid = { ... }

Perform __Hyperparameter Tuning__ with Cross-Validation (GridSearchCV)

In [43]:
# --- GridSearchCV for RandomForestRegressor ---
print("\n--- Starting GridSearchCV for RandomForestRegressor ---")
rf_grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_param_grid,
    scoring=mae_scorer,
    cv=5,            # 5-fold cross-validation
    n_jobs=-1,       # Use all available CPU cores
    verbose=1        # Show progress
)
rf_grid_search.fit(X_train, y_train)

print("\nBest parameters for RandomForestRegressor:", rf_grid_search.best_params_)
print("Best cross-validated MAE for RandomForestRegressor:", -rf_grid_search.best_score_)


# --- GridSearchCV for GradientBoostingRegressor ---
print("\n--- Starting GridSearchCV for GradientBoostingRegressor ---")
gb_grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=gb_param_grid,
    scoring=mae_scorer,
    cv=5,
    n_jobs=-1,
    verbose=1
)
gb_grid_search.fit(X_train, y_train)

print("\nBest parameters for GradientBoostingRegressor:", gb_grid_search.best_params_)
print("Best cross-validated MAE for GradientBoostingRegressor:", -gb_grid_search.best_score_)


--- Starting GridSearchCV for RandomForestRegressor ---
Fitting 5 folds for each of 27 candidates, totalling 135 fits

Best parameters for RandomForestRegressor: {'max_features': 0.6, 'min_samples_leaf': 2, 'n_estimators': 100}
Best cross-validated MAE for RandomForestRegressor: 3.654115476592851

--- Starting GridSearchCV for GradientBoostingRegressor ---
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameters for GradientBoostingRegressor: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}
Best cross-validated MAE for GradientBoostingRegressor: 3.598239884492996


 Evaluate the Best Models on the Test Set


In [46]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import joblib # For model persistence

print("--- Step 5: Evaluate the Best Models on the Test Set ---")

# Retrieve the best models from GridSearchCV
best_rf_model = rf_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_

# Make predictions on the test set
rf_predictions = best_rf_model.predict(X_test)
gb_predictions = best_gb_model.predict(X_test)

print("\n--- RandomForestRegressor Evaluation ---")
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))

print(f"RandomForestRegressor MAE on Test Set: {rf_mae:.4f}")
print(f"RandomForestRegressor R2 on Test Set: {rf_r2:.4f}")
print(f"RandomForestRegressor RMSE on Test Set: {rf_rmse:.4f}")

print("\n--- GradientBoostingRegressor Evaluation ---")
gb_mae = mean_absolute_error(y_test, gb_predictions)
gb_r2 = r2_score(y_test, gb_predictions)
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_predictions))

print(f"GradientBoostingRegressor MAE on Test Set: {gb_mae:.4f}")
print(f"GradientBoostingRegressor R2 on Test Set: {gb_r2:.4f}")
print(f"GradientBoostingRegressor RMSE on Test Set: {gb_rmse:.4f}")

# You can also compare them:
print("\n--- Model Comparison ---")
if rf_mae < gb_mae:
    print("RandomForestRegressor performed better in terms of MAE.")
elif gb_mae < rf_mae:
    print("GradientBoostingRegressor performed better in terms of MAE.")
else:
    print("Both models have similar MAE.")

# Optional: Store evaluation metrics for later analysis if needed
evaluation_results = {
    'RandomForestRegressor': {'MAE': rf_mae, 'R2': rf_r2, 'RMSE': rf_rmse},
    'GradientBoostingRegressor': {'MAE': gb_mae, 'R2': gb_r2, 'RMSE': gb_rmse}
}

--- Step 5: Evaluate the Best Models on the Test Set ---

--- RandomForestRegressor Evaluation ---
RandomForestRegressor MAE on Test Set: 3.7873
RandomForestRegressor R2 on Test Set: 0.3737
RandomForestRegressor RMSE on Test Set: 4.7672

--- GradientBoostingRegressor Evaluation ---
GradientBoostingRegressor MAE on Test Set: 3.6403
GradientBoostingRegressor R2 on Test Set: 0.4217
GradientBoostingRegressor RMSE on Test Set: 4.5808

--- Model Comparison ---
GradientBoostingRegressor performed better in terms of MAE.


Select the Final Model and Model Persistence

In [47]:
import joblib
import os
from sklearn.ensemble import GradientBoostingRegressor

print("--- Model Selection ---")

# Based on evaluation results, GradientBoostingRegressor is the chosen model.
# Assuming gb_grid_search is defined and available from previous cells in the notebook
best_model_to_save = gb_grid_search.best_estimator_ # Access the best estimator from GridSearchCV
model_name = "GradientBoosting"

print(f"\nSelected final model: {model_name}")
print("Best parameters for selected model:")
print(best_model_to_save.get_params())


# Model Persistence (Saving the Model)
# Define the relative path to the 'models' directory
# This assumes the notebook is run from within the 'notebooks' directory
# and 'models' is a sibling directory to 'notebooks'
model_dir = os.path.join('..', 'models')

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Construct the full path for the model file
model_filename = os.path.join(model_dir, f'{model_name}_F1_Race_Predictor_model.joblib')

joblib.dump(best_model_to_save, model_filename, compress=3)

print(f"\nModel saved successfully to: {model_filename}")

# Loading the Model to Verify
print("\n--- Verifying Model Loading (Optional) ---")
loaded_model = joblib.load(model_filename)
print(f"Model loaded successfully: {loaded_model}")

--- Model Selection ---

Selected final model: GradientBoosting
Best parameters for selected model:
{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}

Model saved successfully to: ..\models\GradientBoosting_F1_Race_Predictor_model.joblib

--- Verifying Model Loading (Optional) ---
Model loaded successfully: GradientBoostingRegressor(learning_rate=0.05, random_state=42)
